### Exiting gracefully from an error



You can make Python exit gracefully if you prepare for well-known
errors, e.g. an `ImportError` if a library is not installed. The code
below will only work in IPython - otherwise you need a

    # Install pint if necessary
    try:
        import pint
    except ImportError:
        !pip install pint



### Download and install `modsim`



The `download` function loads a Python file from `url`. It also will exit
gracefully if there is no file at the URL. We then call the function
on the location of `modsim` and import its functions:



In [1]:
# download modsim.py if necessary
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

# call function for download
download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/modsim.py')

# import functions from modsim
from modsim import *

Modify this last command so that Python exits in case `modsim` is not
installed:



In [1]:
# Install pint if necessary
try:
    import modsim
except ImportError:
    print("modsim is not installed.")

### Bike sharing model



#### The simulation sandbox



Consider a bike sharing system for students traveling between two
sites, LEAP on Lyon's campus, and the Batesville City Community Centre
on 20th Street.
![img](./img/bikeshare_locations.png)



#### System description



The **system** contains 12 bikes as **elements** and 2 bike racks each with a
capacity to hold 12 bikes.

**State** changes in either location are caused by students checking out
bikes at one and riding to the other location.

In the **simulation**, we keep track on where the bikes are using the
`modsim.State` function:

1.  import the `modsim` library
2.  look at the `help` for `State`



In [1]:
import modsim
# help(State)

#### More about the `State` function



`State` is defined with `**variables` as argument, which means that any
keyword arguments passed to that function will be collected into a
dictionary called `variables`.

This means that you can initialize `State` with any number of keyword
variables. For example, you could use the function to represent a
simple bank account:



In [1]:
from modsim import State
bank_account = State(balance=100,interest_rate=0.05)
print(f'You have ${bank_account.balance} in your \
{int(bank_account.interest_rate*100)}% interest bank account.')

You have $100.0 in your 5% interest bank account.

When you check the `type` of `modsim.State()`, you can see that it is
based on a `pandas` `Series` object, or one-dimensional `numpy` array, or a
vector ([see doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)):



In [1]:
print(type(bank_account))

<class 'pandas.core.series.Series'>

You can also use `source_code` to see the code for the function:



In [1]:
source_code(State)

def State(**variables):
    """Contains the values of state variables."""
    return pd.Series(variables, name='state')

`Series` objects provide their own `plot` function, `Series.plot()`.



#### Using `State` to describe the system



We store the state of the bike sharing system in a state `bikeshare`,
with the number of bike in either location:



In [1]:
bikeshare = State(leap=10, city=2)

We can now get the value of the state variables `leap` and `city`:



In [1]:
print(f'Bikes at LEAP: {bikeshare.leap}')
print(f'Bikes at Community Center: {bikeshare.city}')

Bikes at LEAP: 10
Bikes at Community Center: 2

To see all state variables and their values, just enter the object's
name (this is better formatted in IPython):



In [1]:
print(bikeshare)

leap    10
city     2
Name: state, dtype: int64

#### Updating the state of the system



To update the system, we can either assign new values to the state
variables, or we can use C-style update operators `+=` and `-=`:



In [1]:
bikeshare.leap = 9
bikeshare.city = 3
print(bikeshare)

leap    9
city    3
Name: state, dtype: int64

We use the update operators to return the system to the previous state:



In [1]:
bikeshare.leap += 1
bikeshare.city -= 1
print(bikeshare)

leap    10
city     2
Name: state, dtype: int64

The last line of the printout are `Series` metadata. To lose them but
retain the tabular format, loop over the items:



In [1]:
print(f'     {bikeshare.name}')
for index, value in bikeshare.items():
    print(f'{index}     {value}')

state
leap     6
city     6

#### Defining functions



To be able to reuse code, we put it into functions. In Python, the
template to create a function named `foo` (without arguments) that
returns nothing looks like this:

    def foo():
        # do something
        return #something

A simple example:



In [1]:
# define function
def hello():
    print("hello there")

# call function
hello()

hello there

The usual sins apply: all variables inside the function are local, and
if you want to return something to the calling routine, you need
`return`:



In [1]:
# define function with return value
def hello_again():
    msg = "Greeting complete."
    print("hello again")
    return msg

# call function and print return value
returned_msg = hello_again()
print(returned_msg)

hello again
Greeting complete.

Run this through pythontutor.com to see what happens when ([solution](https://pythontutor.com/render.html#code=%23%20define%20function%20with%20return%20value%0Adef%20hello_again%28%29%3A%0A%20%20%20%20msg%20%3D%20%22Greeting%20complete.%22%0A%20%20%20%20print%28%22hello%20again%22%29%0A%20%20%20%20return%20msg%0A%0A%23%20call%20function%20and%20print%20return%20value%0Areturned_msg%20%3D%20hello_again%28%29%0Aprint%28returned_msg%29&cumulative=false&curInstr=8&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)).



#### Defining an updating function



Rather than repeat the update every time a bike moves, define a
function that reflects a move of a bike from LEAP to the Community
Center, `bike_to_city`:



In [1]:
def bike_to_city():
    bikeshare.leap -= 1
    bikeshare.city += 1

Now print the current state, then update it, then print the new state:



In [1]:
print(bikeshare)
bike_to_city()
print(bikeshare)

leap    10
city     2
Name: state, dtype: int64
leap    9
city    3
Name: state, dtype: int64

There's nothing that keeps our bike share state variables from going
outside of the [0,12] range:

1.  Write a function `reset` that restores a particular state, e.g. with
    6 bikes in either location.
2.  The function should print the old and the new state.
3.  The function should announce itself "System reset".
4.  Run `bike_to_city` a few times until the values are wrong.
5.  Restore the steady state using your new function.



In [1]:
def reset():
    print("System reset. Old state:")
    print(bikeshare)
    bikeshare.leap = 6
    bikeshare.city = 6
    print("New state:")
    print(bikeshare)

Testing:



In [1]:
print(bikeshare)
bike_to_city()
bike_to_city()
bike_to_city()
bike_to_city()
bike_to_city()
reset()

Alter the `bike_to_city` function and print out "Moving bike to city"
every time the function is called, test the function, and then move
the system back to the steady state.



In [1]:
def bike_to_city():
    print("Moving bike to city.")
    bikeshare.leap -= 1
    bikeshare.city += 1

bike_to_city()
reset()

Moving bike to city.
System reset. Old state:
leap    5
city    7
Name: state, dtype: int64
New state:
leap    6
city    6
Name: state, dtype: int64

#### Pseudorandom generator



As a simple model of customer behavior within the system, we use a
pseudorandom number generator to determine when customers arrive at
each bike station.

The function `modsim.flip` generates random coin tosses, i.e. it
simulates tosses of a fair coin with default probability 0.5 for
either side, and returns a Boolean value, `True` or `False`.

It is based on NumPy's `random` function:



In [1]:
source_code(flip)

def flip(p=0.5):
    """Flips a coin with the given probability.
:
    p: float 0-1
:
    returns: boolean (True or False)
    """
    return np.random.random() < p

Call the function with a probability between 0 and 1, e.g. 70%. On
average, it will return `True` with probability 70% or `False` with
probability 30%:



In [1]:
for _ in range(10):
    print(flip(0.7),end=" ")

True True True False True False True False True True

To control program behavior with Boolean values, we use conditional
statements. The general form of such a statement is as follows:



In [1]:
if condition:
    # do something if condition is True
else:
    # do something else if condition is False

The following program simulates a fair coin: it prints "heads" if the
`flip` results in `True`, and "tails" if it results in `False`.



In [1]:
if flip(0.5):
    print("heads")
else:
    print("tails")

heads

For the particular argument 0.5 we could have left the argument out
since `flip` is defined as `flip(p=0.5)` as we saw earlier.



#### Simulating customers as coin tosses



We can use `flip` to simulate the arrival of customers who want to
borrow a bike: If customers arrive at the LEAP station every two
minutes on average, then the chance of an arrival during any
one-minute period is 50%:



In [1]:
if flip(0.5):
    bike_to_city()

Moving bike to city.

If customers arrive at the Community Center station every three
minutes on average, the chance of an arrival during any one-minute
period is 33%:



In [1]:
if flip(0.5):
    bike_to_city()

**Challenge:** what is the mathematical formulation for the relationship
that leads to this "chance of arrival in any one-minute period is N
percent"? (Tip: the "arrival" is called a "draw with replacement".)

Both of these snippets together with functions that change the state
of the system can be used to simulate a time step - in this case one
minute:



In [1]:
def step():
    if flip(0.5):
        bike_to_city()
    if flip(0.33):
        bike_to_leap()

Depending on the random results from `flip`, a `step` moves a bike to the
Community Centre or to the LEAP bike station, or neither, or both.

Before you can try it, you need to remember how to move a bike and
create the function `bike_to_leap()`:



In [1]:
def bike_to_leap():
    print("Moving bike to LEAP.")
    bikeshare.leap += 1
    bikeshare.city -= 1

Simulating customers and bikes:



In [1]:
step()

Moving bike to city.
Moving bike to LEAP.